<a href="https://colab.research.google.com/github/RaghavMaskara21/Machine-Learning/blob/main/5_method_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
!pip install emoji
import re
import nltk
import emoji
import string
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [18]:
spam = pd.read_csv("spam.csv", encoding = "latin-1")
spam.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [19]:
def cleaner(text):
  text = re.sub("@[A-Za-z0-9]+","",text)
  # Remove @user sign
  text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text)
  # Remove http links
  text = " ".join(text.split())
  text = ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)
  # Remove Emojis
  text = text.replace("#", "").replace("_", " ")
  # Remove hashtag sign but keep the text
  text = " ".join(w for w in nltk.wordpunct_tokenize(text) \
  if w.lower() in words or not w.isalpha())
  text = re.sub("[^-9A-Za-z ]", "" , text)
  # Remove punctuations
  text = "".join([i.lower() for i in text if i not in
  string.punctuation]) #Case normalisarion
  return text
spam['cleaned'] = spam['v2'].apply(cleaner)
spam.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,cleaned
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,go until point crazy available only in n gre...
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,lar u
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry in a to win fa cup final st may ...
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,u dun say so early u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,i don t think he goes to he around here though


In [20]:
nltk.download('wordnet')
# w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
  return [lemmatizer.lemmatize(w,pos = 'v') for w in text.split()]
spam['lemmatize_text'] = spam['cleaned'].apply(lemmatize_text)
spam.head(5)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,cleaned,lemmatize_text
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,go until point crazy available only in n gre...,"[go, until, point, crazy, available, only, in,..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,lar u,"[lar, u]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry in a to win fa cup final st may ...,"[free, entry, in, a, to, win, fa, cup, final, ..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,u dun say so early u c already then say,"[u, dun, say, so, early, u, c, already, then, ..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,i don t think he goes to he around here though,"[i, don, t, think, he, go, to, he, around, her..."


In [21]:
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(tokens):
  tokens=[word for word in tokens if word not in stopword]
  return tokens
spam['cleaned_text'] = spam['lemmatize_text'].apply(remove_stopwords)
spam.head(3)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,cleaned,lemmatize_text,cleaned_text
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,go until point crazy available only in n gre...,"[go, until, point, crazy, available, only, in,...","[go, point, crazy, available, n, great, world,..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,lar u,"[lar, u]","[lar, u]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry in a to win fa cup final st may ...,"[free, entry, in, a, to, win, fa, cup, final, ...","[free, entry, win, fa, cup, final, st, may, te..."


In [22]:
nltk.download('averaged_perceptron_tagger')
pos_tags = []

for token in spam['cleaned_text']:
  pos_tag = nltk.pos_tag(token)
  pos_tags.append(pos_tag)

nouns_array = []
no_of_noun = []
verbs_array = []
no_of_verbs = []
adj_array = []
no_of_adj = []
for token in pos_tags:
  noun = []
  verb = []
  adj = []
  for i in token:
    if(i[1]=="NN" or i[1]=="NNS" or i[1]=="NNP" or i[1]=="NNPS"):
      noun.append(i[0])
    if(i[1]=="JJR" or i[1]=="JJS"):
      adj.append(i[0])
    if(i[1]=="VB" or i[1]=="VBG" or i[1]=="VBD" or i[1]=="VBN" or i[1]=="VBP" or i[1]=="VBZ"):
      verb.append(i[0])
  nouns_array.append(noun)
  no_of_noun.append(len(noun))
  verbs_array.append(verb)
  no_of_verbs.append(len(verb))
  adj_array.append(adj)
  no_of_adj.append(len(adj))

spam['nouns'] = nouns_array
spam['no_of_nouns'] = no_of_noun
spam['verbs'] = verbs_array
spam['no_of_verbs'] = no_of_verbs
spam['adjectives'] = adj_array
spam['no_of_adj'] = no_of_adj
one_letter_word_count= []
two_letter_word_count= []
def select_one_letter_word(word):
  if(len(word)==1):
    return True
  return False

def select_two_letter_word(word):
  if(len(word)==2):
    return True
  return False

for token in spam['cleaned_text']:
  one_letter = list(filter(select_one_letter_word, token))
  two_letter = list(filter(select_two_letter_word, token))
  one_letter_word_count.append(len(one_letter))
  two_letter_word_count.append(len(two_letter))

spam["one_letter_word_count"] = one_letter_word_count
spam["two_letter_word_count"] = two_letter_word_count
spam.to_csv('spam_features.csv')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
